## Description

Notebook playing around with converting radial integrals to gaussian as a preprocessing step.

## Imports

In [ ]:

%load_ext autoreload
%autoreload 2

import sys
import os
from pathlib import Path
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from sklearn.linear_model import Lasso
from scipy.integrate import solve_ivp, odeint
import numpy as np
from scipy.sparse import diags
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import pysindy as ps
import pickle

# Ignore matplotlib deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Seed the random number generators for reproducibility
np.random.seed(100)

integrator_keywords = {}
integrator_keywords['rtol'] = 1e-12
integrator_keywords['method'] = 'LSODA'
integrator_keywords['atol'] = 1e-12

# Update path to include mypkg
sys.path.insert(0, str(Path(os.path.abspath('')).parent.parent.absolute()))

from src import helpers, plot_data, global_config, datasets
config = global_config.config
image_dir_og = config.jpg_dir

%matplotlib inline



## Define gaussian

In [ ]:
def gaussian(x, mu, sigma, a):
    return a * 1.0 / (np.sqrt(2.0 * np.pi) * sigma) * np.exp(-np.power((x - mu) / sigma, 2.0) / 2)

## Load data (and smooth it)

In [ ]:

# Load the data
dataset_path = "MNIST_Test"
dataset_name = "MNIST"
config.top_dir = str(Path(image_dir_og).parent / "fft_images" / dataset_path)
file = os.path.join(config.top_dir, "pkl", f"{dataset_name}_og_integral.pkl")
with open(file, 'rb') as file:
    d = pickle.load(file)


In [ ]:

# Load the data stored in a matlab .mat file
u = helpers.moving_average(d['ints'],n=5,axis=0)
u = u[0:150,:]
t = np.arange(u.shape[0])
x = d['r_x']
u = u.T

dt = t[1] - t[0]
dt = dt.astype(float)
dx = x[1] - x[0]
dx = float(dx)

means = list()
stdevs = list()
gaussians = list()

# Fit gaussians
for idx in range(u.shape[1]):
    ux = u[:,idx]
    popt, pcov = curve_fit(gaussian, x, ux)
    mu = popt[0]
    sigma = popt[1]
    amplitude = popt[2]

    means.append(mu)
    stdevs.append(sigma)
    gaussians.append(gaussian(x, mu, sigma, amplitude))

means = np.asarray(means)
stdevs = np.asarray(stdevs)
gaussians = np.stack(gaussians)
gaussians = gaussians.T

print(u.shape)
print(gaussians.shape)

# Plot u, fitted gaussians, mean, and stdev

nrows = 3
ncols = 2

plt.figure(figsize=(4*ncols, 4*nrows))
plt.subplot(nrows, ncols, 1)
plt.pcolormesh(t, x, u)
plt.xlabel('t', fontsize=16)
plt.ylabel('x', fontsize=16)
plt.title(r'$u(x, t)$', fontsize=16)

plt.subplot(nrows, ncols, 2)
plt.pcolormesh(t, x, gaussians)
plt.xlabel('t', fontsize=16)
plt.ylabel('x', fontsize=16)
plt.title(r'gaussians', fontsize=16)

plt.subplot(nrows, ncols, 3)
plt.plot(x, u[:,-1], 'k')
plt.xlabel('t', fontsize=16)
plt.ylabel('x', fontsize=16)
plt.title(r'$u(x, t=-1)$', fontsize=16)

plt.subplot(nrows, ncols, 4)
plt.plot(x, gaussians[:,-1], 'r')
plt.xlabel('t', fontsize=16)
plt.ylabel('x', fontsize=16)
plt.title(r'corresponding gaussian', fontsize=16)

plt.subplot(nrows, ncols, 5)
plt.plot(t, means, 'r')
plt.fill_between(t, means+stdevs, means-stdevs, facecolor='red', alpha=0.25)
plt.xlabel('t', fontsize=16)
plt.ylabel('x', fontsize=16)
plt.title(r'gaussian means', fontsize=16)

plt.subplot(nrows, ncols, 6)
plt.xlabel('t', fontsize=16)
plt.ylabel('x', fontsize=16)

colors = plt.cm.plasma(np.linspace(0.,1.,u.shape[1]))
for i, color in enumerate(colors):
    plt.plot(x, u[:,i], label=f"$u_{{{i}}}$", color=color)
plt.ylim(auto=True)

plt.title(r'u(x,t)', fontsize=16)

plt.tight_layout()
plt.show()
plt.close()